# Computing with word embeddings: essential methods and operations

| Author | Last update |
|:------ |:----------- |
| Hauke Licht (https://github.com/haukelicht) | 2025-09-23 |

<br>
<a target="_blank" href="https://colab.research.google.com/github/haukelicht/advanced_text_analysis/blob/main/notebooks/embedding/word_embedding_basics.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook illustrates how to use `gensim` to compute with word vectors (e.g., word2vec).

We'll cover how to

- compute two words similarity
- find the most similar words for a focal word
- solve analogy tasks

<span style="color: red">*IMPORTANT:*</span>
This notebook contains exercises that are marked with the keyword pattern `# TODO:`
In cells containing this keyword, you _first_ **add** the necessary code to be able to successfully run the cell.

For example, the following code won't run without completing the TODO:

```python
for i in range(4):
    # TODO: add logic here!
```

## Setup

#### Colab

In [ ]:
# check if on colab
COLAB = True
try:
    import google.colab
except:
    COLAB=False

if COLAB:
    # shallow clone of current state of main branch 
    !git clone --branch main --single-branch --depth 1 --filter=blob:none https://github.com/haukelicht/advanced_text_analysis.git
    
    !pip install gensim

#### Load required packages

Let's load modules we'll need throughout the notebook here.

<!-- **_Note:_** I asume you have follewed the setup instructions at https://github.com/haukelicht/advanced_text_analysis/tree/main/setup -->

In [ ]:
# file in- and export
import os

# for working with word embeddings
import gensim

# for using arrays and data frames
import pandas as pd
import numpy as np

# for plotting
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
plt.style.use('ggplot')

## 1. Loading a pre-trained word2vec model

In this notebook, we will use a word2vec model that has been trained on the *Google News Corpus*.

It is downloadable from the internet and we use the `gensim` downloader API to do this.

In [ ]:
# load a pre-trained word2vec model (shipped with gensim)
import gensim.downloader as api

# load the model and name it's instance in our notebook environment 'word2vec'
model = api.load('word2vec-google-news-300')

**_Note:_** The above code will download the model (if not already done so) to the folder specified in `api.BASE_DIR`:

In [ ]:
# show path
print(api.BASE_DIR)

# list files in folder
os.listdir(api.BASE_DIR)

Alternatively, you could load an already downloaded model from your computer:

```python
from gensim.models import KeyedVectors

model_name = 'word2vec-google-news-300'
model_dir = os.path.join(api.BASE_DIR, model_name)
model_path = os.path.join(model_dir, model_name + '.gz') 
model = KeyedVectors.load_word2vec_format(model_path, binary=True)
```

### 1.1 Understanding the `word2vec` object

Let's inspect the **`word2vec`** object we have created so you understand how it's structured, and what **attributes** and **methods** it has.

The word embedding model is an instance of the `gensim.models.keyedvectors.KeyedVectors'` class.

In [ ]:
print(type(model))

You can view the documentation with `?gensim.models.keyedvectors.KeyedVectors`.

In [ ]:
# list all attributes
list(model.__dict__.keys())

The most relevant attributes are

- `vector_size`: number of dimensions of embeddings 
- `index_to_key`: pyhton list listing words' indexes in the embedding matrix
- `key_to_index`: pyhton list listing words in the embedding matrix
- `vectors`: 2-dimensional `numpy` array recording word embeddings

create a vocabulary-index-to-word mapping


In [ ]:
dict(enumerate(model.key_to_index))

Let's loop at the `vectors` that store the word embeddings:

In [ ]:
# shape of the embedding matrix (number of words x number of embedding dimensions)
model.vectors.shape

In [ ]:
# how many storage/RAM needed for this matrix?
n_bytes = model.vectors.nbytes
gb = n_bytes / (1024**3)
print(round(gb, 2), "GB")

### 1.2 Essential methods available for `KeyedVectors` objects

`gensim`'s  `KeyedVectors` class comes with a lot of useful pre-defined methods.

Let's start with different ways of accessing a word's embedding.

#### (a) Accessing a word's embedding

In [ ]:
# OPTION 1: directly index the object with a string value
print(model['good'][:10]) # <== look only at first 10 dimensions
len(model['good'])

In [ ]:
# OPTION 2: use the get_vector() method
model.get_vector('good')[:10]

#### (b) Finding where a word's embedding is located

`model.key_to_index` maps the "words" (tokens) in the model's vocabulary to their index position in the emebdding matrix.

In [ ]:
model.key_to_index

So an alternative way to access a word's embedding is 

1. using the model's `key_to_index` attribute to lookup a word's index in the embedding matrix, and
2. index the model's embedding matrix accordingly

In [ ]:
idx = model.key_to_index['good']
print(idx)
model.vectors[idx][:10] # <== just print first 10 dimensions

### 1.3 Visualizing word vectors

Since word embeddings have too many dimensions to display their locations, we usually apply dimensionality reduction techniques.

In this notebook, we'll simply rely on *Principal Component Analysis* (PCA).
Belo is some code that illustrates this.

**_Note:_** There are better options like UMAP or t-SNE that do a better job in preserving relative distances between data points when reducing dimensionality.



In [ ]:
words = [
    # beverages
    'coffee', 'tea', 'beer', 'wine', 'brandy', 'rum', 'champagne', 'water',
    # animals
    'dog', 'horse', 'cat', 'monkey', 'parrot', 'koala', 'lizard', 'frog', 'monkey', 'ape', 'kangaroo', 'wolf',
    # countries
    'france', 'germany', 'hungary', 'luxembourg', 'australia', 'fiji', 'china',
    # institutions
    'parliament', 'government', 'opposition', 'coalition', 'prime_minister', 'minister',
]

X = np.array([model[w] for w in words if w in model.key_to_index])

pcs = PCA(n_components=2).fit_transform(X)

plt.scatter(pcs[:,0], pcs[:,1], c='r', edgecolors='k', alpha=.4)
for word, (x, y) in zip(words, pcs):
    plt.text(x+0.05, y+0.05, word)

## 2. Essential computations

In the next subsections, we will learn how use `gensim`'s `KeyedVectors` objects to 

1. compute similarities, 
2. find nearest neighbors, and 
3. solve analogy tasks.

**_Note:_** 
We will only go through the methods that will be required to complete exercises in our course.
Visit Radim Řehůřek's [summary](https://radimrehurek.com/gensim/models/keyedvectors.html) for a more comprehensive overview of available methods with excellent examples.


### 2.1 Computing similarities


An central quantity of interest in the study of lexical semantics (i.e., the "meaning" of words) is **word similarity** &mdash; how similar the "meanings" of two words are.

In CSS studies using word embeddings, computating similarities is often an essential computation step.

#### Intuition

In the embedding space, we can **measure two word vectors' similarity** of by their **_angle_**: the extent to which they *point in similar directions* relative to the coordiante origin $(0, 0, \ldots, 0)$.

Say you stand at the origin of a $d$-dimensional coordinate system.
Then asking whether two vectors $\mathbf{u}$ and $\mathbf{v}$ are similar is the same as asking: 

> "Would I end up in a similar location if I follow the direction of vector $\mathbf{u}$ rather than $\mathbf{v}$?"

The answer to this question aligns with the magnitude of the angle:

- If the two vectors are very similar, you'd end up in a similar location (e.g., $\mathbf{u} = [1, 1]$ and $\mathbf{u} = [.8, .9]$)
- If they are pointing in completely opposite directions, you'll end up on the other "side" of the vector space if you follow $\mathbf{u}$ rather than $\mathbf{v}$ (e.g., (e.g., $\mathbf{u} = [1, 1]$ and $\mathbf{u} = [-1, -1]$)

#### Illustration

The below plot visually illustrates this, given the following vectors (i.e., 2D points):

In [ ]:
vecs = [
    [ 1.0,  1.0], # vector a
    [ 0.8,  0.9], # vector b
    [ 0.5,  .2],  # vector c
    [-1.0,  1.0], # vector d
    [-1.0, -1.0], # vector e
]
vecs = np.array(vecs)
vecs

In [ ]:
# plot vectors' values on 1st and 2nd dimension on x and y axes
plt.scatter(vecs[:,0], vecs[:,1])

# add vertical and horizontal coordinate lines running through (0, 0) to the plot
plt.axvline(0, c='grey', alpha=.5)
plt.axhline(0, c='grey', alpha=.5)

# draw lines from the coordinate origin (0, 0) to each point
vec_names = ['a', 'b', 'c', 'd', 'e']
for i, vec in enumerate(vecs):
    plt.annotate(
        '', # no text
        xy=vec, # end point of the line
        xytext=(0, 0), # start point of the line
        arrowprops=dict(facecolor='black', color='black', width=.5, headwidth=3) # arrow style
    )
    plt.text(vec[0], vec[1], f'{vec_names[i]}')

plt.show()


So in this example,

- $\mathbf{a}$ and $\mathbf{b}$ are very similar (because they point in a similar direction)
- $\mathbf{a}$ and $\mathbf{c}$, and $\mathbf{b}$ and $\mathbf{c}$ are also still somewhat similar
- $\mathbf{a}$ and $\mathbf{d}$ are orthorgonal ("unrelated"), and
- $\mathbf{a}$ and $\mathbf{e}$ are maximally dissimilar (because they point in opposite directions)

#### Math

The extent to which they point in similar directions relative to the coordiante origin $(0, 0,
 \ldots, 0)$ can be measured by the **cosine similarity**:

$$
\text{Cosine Similarity} = \frac{{\mathbf{u} \cdot \mathbf{v}}}{{\| \mathbf{u} \| \times \| \mathbf{v} \|}}
$$


$\mathbf{u}$ and $\mathbf{v}$ are the vectors whose similarity is being measured.

- the *numerator* (top) measures the similarity of 
- the *denominator* (bottom) normalizes ("rescales") the similarity measure to range between -1 and 1 

The $\boldsymbol{\cdot}$ operator denotes the **dot product** of two vectors. 
It measures the *similarity in the direction of two vectors.*
(Think of vectors as directions from the origin $(0, 0, \ldots, 0)$).

- The dot product is computed as the sum of the element-wise products of vectors' entries: 
$$
\mathbf{u} \cdot \mathbf{v} = \sum\limits_{i=1}^{d} u_i \times v_i
$$
- $u_i \times v_i$ is high if the two entries are similar (point in the same directio nrelative to the coordinate origin $0$)
- Thus, $\mathbf{u} \cdot \mathbf{v}$ is the higher, the more of their $(u_i, v_i)$ elements are similar.

$\| \mathbf{u} \|$ and $\| \mathbf{v} \|$ represent the **Euclidean norms** (magnitudes) of vectors $\mathbf{u}$ and $\mathbf{v}$.
The magnitude (or length) of a vector measures its *distance from the origin in the  $d$-dimensional space.*
It is computed as 

$$
\| \mathbf{x} \| = \sqrt{ \sum\limits_{i=1}^{d} x_i^2}
$$

Having the $\| \mathbf{u} \| \times \| \mathbf{v} \|$ in the denominator if the cosine similarity function makes the metric scale invariant (i.e., insensitive to the lengths/magnitudes of $\mathbf{u}$ and $\mathbf{v}$).

#### Code (from scratch)

In [ ]:
# define a function to compute the cosine similarity between two word vectors
import numpy as np

def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
    
    Arguments:
        u -- a word vector of shape (n,)
        v -- a word vector of shape (n,)
    
    Returns:
        cosine_similarity -- the cosine similarity between u and v
    """
    
    # compute the dot product between u and v
    dot = np.dot(u, v)
    
    # compute the Euclidean norm of u
    norm_u = np.sqrt(np.sum(np.square(u)))
    # compute the Euclidean norm of v
    norm_v = np.sqrt(np.sum(np.square(v)))
    
    # divide the dot product by the product of the L2 norms
    cosine_similarity = dot / (norm_u * norm_v)
    
    return cosine_similarity

**_Note:_** We implement our own `cosine_similarity` function here to map code to math. Below and in other notebooks, we'll use the `cosine` function from the `scipy.spatial.distance` module.

Let's compute similarties for vectors $\mathbf{b}$-$\mathbf{e}$ in `vecs` to vector $\mathbf{a}$:

In [ ]:
for i in range(1, vecs.shape[0]):
    s = cosine_similarity(vecs[0], vecs[i])
    print(f"a = {vecs[0]}, {vec_names[i]} = {vecs[i]}, cos(a, {vec_names[i]}) = {s:+0.3f}")

**_Note:_** inside `print` above we use python's string formatting syntax to directly insert the variable values we want to print.
It works like this:

Let's apply the same function to compute the similarity of two words in our `word2vec` embeddings:

compute cosine similarity for two word vectors in our embedding model:

In [ ]:
cosine_similarity(model['good'], model['bad'])

**_Note:_** `scipy` implements the cosine distiance metric for us ;)

In [ ]:
from scipy.spatial.distance import cosine

1-cosine(model['good'], model['bad'])

#### Code (using `gensim``)

But our `word2vec` instance of `gensim`'s `KeyedVectors` class also comes with a built-in method to compute two words' similarity:

In [ ]:
# compute the cosine similarity between two word embeddings in the word2vec object
model.similarity('good', 'bad')

### Exercise 1 👩‍💻

Let's implement a classic approach to evaluate how word embeddings capture cultural biases in their training copora.
Here, we'll focus on **_gender bias_** &mdash; the differential association of traits and attributes with women and men (my lose definition).

Compile a list `comparison_words` with occupations, character traits, and other words that might exhibit gender bias.
Then compute how similar each word is with terms like 'man' and 'women', that indicate the male and female genders.

Which words exhibit gender bias?
And in which direction? 
Do you spot a pattern?

In [ ]:
comparison_words = [
    'programmer',
    'scientist',
    'smart',
    'emotional',
    'caring',
    # TODO: add more interesting words here
]

In [ ]:
# compute similarities to male and female terms
male_terms = ['man']
female_terms = ['woman']

for word in comparison_words:
    # TODO: implement logic

In [ ]:
# TODO: summarize the results in a table or figure
# hint: if you have more than one term per gender, you might want to compute the average of comparison term--gender word similarities

### 2.2. Finding most similar terms (nearest neighbors)


Since word vectors capture similarities in words co-occurence patterns, their locations in the embedding space can be use to find lists of semantically and functionally similar words.

#### Intuition

In the embedding space, searching for similar words for a focus word *w* can be implemented as [*nearest neighbors* search](https://en.wikipedia.org/wiki/Nearest_neighbour_algorithm) because words pairs' distances measure their similarity.

#### Illustration

Let's use the same 2-dimensional vectors as in the example above.

In [ ]:
vecs = [
    [ 1.0,  1.0], # vector a
    [ 0.8,  0.9], # vector b
    [ 0.5,  .2],  # vector c
    [-1.0,  1.0], # vector d
    [-1.0, -1.0], # vector e
]
vecs = np.array(vecs)

In [ ]:
# plot vectors' values on 1st and 2nd dimension on x and y axes
plt.scatter(vecs[:,0], vecs[:,1])

# draw lines from the coordinates of vector a to vectors b, c, d, and e
vec_names = ['b', 'c', 'd', 'e']
for i, vec in enumerate(vecs[1:]):
    plt.annotate(
        '', # no text
        xy=vec, # end point of the line
        xytext=vecs[0], # start point of the line
        arrowprops=dict(facecolor='black', color='black', width=.5, headwidth=3) # arrow style
    )
    plt.text(vec[0], vec[1], f'{vec_names[i]}')

plt.show()

#### Code

So we can find out what's the most similar vector to $\mathbf{a}$ by 

1. computing all vectors ($\mathbf{b}$-$\mathbf{e}$) similarity with $\mathbf{a}$, and 
2. selecting the one with the highest similarity among them

In [ ]:
sims = {nm: cosine_similarity(vecs[0], vec) for nm, vec in zip(vec_names, vecs[1:])}
print(sims)

# get name of most similar values (by sorting)
print([nm for nm, s in sorted(sims.items(), key=lambda item: item[1], reverse=True)][0])

# same with numpy
vec_names[np.array(sims.values).argmax()]

Again, `gensim`'s `KeyedVectors` class provides dedicated methods for nearest neighbors-based similar word search:

In [ ]:
# show the five most similar words to 'love'
model.most_similar('mother', topn=10)

In [ ]:
[w for w, _ in model.most_similar('love', topn=5)]

**_Note:_** The `most_similar()` method returns a list of tuples.
Each tuple is a pair of word and cosine similarity score.

Interestingly, among the five terms most similar to 'love', there is also it's [*antonym*](https://en.wikipedia.org/wiki/Opposite_(semantics)) 'hate' (i.e., opposite word).

This illustrates that "similar" in the context of word embeddings does *not* necessarily mean [*synonym*](https://en.wikipedia.org/wiki/Synonym) (i.e.. equal in meaning).
Instead, word embeddings capture a wide range of lexical semantic relations

- synonyms
- antonyms
- hypo- and hypernym relations (subtyoe relations): https://en.wikipedia.org/wiki/Hyponymy_and_hypernymy
- mero- and hoponym relations (part-of relations): https://en.wikipedia.org/wiki/Meronymy_and_holonymy

### Exercise 2 👩‍💻

Let's use nearest neighbors search to find conceptually equivalent terms for a "seed" word.

**_Note:_** This is a typical task in expanding keyword lists for dictionaries.

You can choose which seed word you want to start with (see example below for a suggestion).
But while going through nearest neighbors, keep track of how many of the candidate terms in the top-20 or top-50 terms (or so) you would inlcude in your dictionary, and how many you would discard!

**_Example_**: 
Say you want to compile a dictionary that contains typical words used to express *positive emotions*.
In this case, you could start with the seed word 'happy.'

In [ ]:
[w for w, s in model.most_similar('happy', topn=20)]

In [ ]:
# EXAMPLE: I made decisions
'glad'  # approved
'pleased', # approved
'ecstatic', # approved
'overjoyed', # approved
'thrilled', # approved
'satisfied', # approved
'proud', # approved
'delighted', # approved
'disappointed', # not approved <== !!!
'excited', # approved
'happier', # approved
'Said_Hirschbeck', # not approved <== !!!
# ... and so on

In [ ]:
emo_word = # TODO: pick another emotion word, list the 20 most similar words, and decide which ones you would approve as well
[w for w, s in model.most_similar(emo_word, topn=20)]

### 2.3. Computing centroids

Just as we use keywords in dictionaries to indicate concepts (e.g., positive affect), 
we can use bags of word vectors to "locate" concepts in the embedding space.

Because each indicator word's embedding might caputure a concept only partially, researchers often **average multiple words' embeddings into _centroids_**.

This is as simple as computing the mean of vectors value for each embedding dimension.
And since word vectors in `gensim` are just numpy arrays, it takes no more than two lines of code:

In [ ]:
# take example of words used to indicate concept 'positive sentiment'
positive_sentiment_words = [
    'glad',
    'pleased',
    'ecstatic',
    'overjoyed',
    'thrilled',
    'satisfied',
    'proud',
    'delighted',
    'excited',
    'elated',
    'thankful',
    'enthused',
    'chuffed',
    'grateful',
    'confident',
]

# get word vectors for words in list
vecs = [model[w] for w in positive_sentiment_words if w in model.key_to_index]
len(vecs) > 0 # <== always ensure that this has more than 0 elements 

In [ ]:
vecs[0].shape

In [ ]:
vecs = np.array(vecs)
vecs.shape

In [ ]:
# average them dimension-wise
centroid = np.average(vecs, axis=0)

# this results in a single 300-dimensional vector
centroid.shape

In [ ]:
# there is again a pre-implemented method
centroid = model.get_mean_vector(keys=positive_sentiment_words)

In [ ]:
# NOTE: also useful to compute "sentence embedding"
sentence = "Just write a random sentence."
words = sentence.split(' ')
words = [w.replace('.', '').lower() for w in words ]
sentence_embedding = model.get_mean_vector(keys=words) # <== you should at least re-weight by words tf-idf values!
# but see: https://openreview.net/pdf?id=SyK00v5xx (but the paper is suuuuuuuper old!)
sentence_embedding.shape


In [ ]:
for word in words:
    if word not in model:
        print(word, ': -/-')
    else:
        print(word, ':', cosine_similarity(sentence_embedding, model[word]))

In [ ]:
# combine original values with centroid
vecs = np.concatenate([vecs, centroid[np.newaxis, :]])

# reduce embeddings to 2 dimensions
X = np.array(vecs)
pcas = PCA(n_components=2).fit_transform(X)

# plot
plt.scatter(pcas[:-1,0], pcas[:-1,1], c='black')
plt.scatter(pcas[-1,0], pcas[-1, 1], c='red')
plt.text(pcas[-1,0], pcas[-1, 1], 'centroid')
plt.show()

#### Excurse: Using centroids to finding mismatching terms (the "odd one out")

We can use centroids to find the most mismatching term in a lits of terms.

For this, we 

1. compute the centroid of all  but the focus term's embeddings,
2. compute the similarity of the focus term's embedding and the centroids,
3. repeat this leave-one-out procedure for all terms, and
4. label the one with the lowest to-centroid similarity as 'mismatching'

In [ ]:
terms = [
    'cat',
    'dog',
    'hamster',
    'flower',
    'table'
]

In [ ]:
# apply a principal component analysis to visualize
X = np.array([model[w] for w in terms])
pcs = PCA(n_components=2).fit_transform(X)
plt.scatter(pcs[:,0], pcs[:,1])
# label the points
for i, (x, y) in enumerate(pcs):
    plt.text(x, y, f'{terms[i]}')

In [ ]:
# for one term
c = np.average([model[w] for w in terms[:-1]], axis=0)
terms[-1], cosine_similarity(c, model[terms[-1]])

In [ ]:
# for all terms
sims = list()
for term in terms:
    c = np.average([model[w] for w in terms if w != term], axis=0)
    s = cosine_similarity(c, model[term])
    sims.append(s)
print(sims)

# find index of lowest value
odd_one_out = terms[np.argmin(sims)]
print(f"the mismatching term is: '{odd_one_out}'")

Luckily, `gensim`'s `KeyedVectors` class again provides dedicated methods for nearest neighbors-based similar word search:

In [ ]:
model.doesnt_match(terms)

### 2.4. Solving analogies



"Well-behaved" word embeddings exhibit the ability to automate much more complex tasks than finding sysnonyms, antonyms, and mismatching terms.

One of word2vec's powers that has attracted a lot of attention when Tomas Mikolov *et al.* released  it ten years ago was their ability to solve analogy tasks.

**Analogy tasks** are simple national language inference problems that prompt a model to infer the Y2 in the following relational statement: 

> `X1 is to Y1 as X2 is to Y2.`

For example, a frequently cited analogy problem is,

> `Man is to woman as king is to _____?`

#### Intuition

One can solve analogy problems with word embeddings by performing applying the following geometric operations:

1. View the statement ` X1 is to Y1 ` as an instruction to move from the location of X1 in the direction of Y1.
2. To get to Y2, take the same direction but start from X2.

The following figure from [Jurafsky & Martin (2023)](https://web.stanford.edu/~jurafsky/slp3/6.pdf) illustrates this idea for the analogy "apple is to tree as grape is to vine:" 

![](./imgs/analogy_problem_jurafsky.png)

#### Math

Let's take the analogy problem

> 'man' is to 'woman' as 'king' is to '_____'?

This can be "solved" by computing with word vectors by performing
the following arithmetic operation with the word vectors of, king, man, and woman:

\begin{align}
\mathbf{v}_\text{man} - \mathbf{v}_\text{woman} &\approx \mathbf{v}_\text{king} - \mathbf{v}_\text{?}\\

\mathbf{v}_\text{king} - \mathbf{v}_\text{man} + \mathbf{v}_\text{woman} &\approx \mathbf{v}_\text{?}
\end{align}

The **closest word vector to** $\mathbf{v}_\text{?}$ is the best model's best guess for completing the analogy problem.

In [ ]:
model.key_to_index['bank']

This works because

- the operation $(\mathbf{v}_\text{woman} - \mathbf{v}_\text{man})$ spans a "gender dimension": $\mathbf{v}_\text{gender}$
- adding $\mathbf{v}_\text{gender}$ to $\mathbf{v}_\text{king}$ means to depart from 'king' and move along the gender dimension into the direction of "femininity"

(see Kozlowski et al., 2019, p. 912, for further examples)


#### Code (from scratch)

In [ ]:
v_king = model['king']
v_man = model['man']
v_woman = model['woman']

v_q = v_king - v_man + v_woman
print(v_q.shape)

model.similar_by_vector(v_q, topn=5)

#### Code (with `gensim`)

In [ ]:
model.most_similar(positive=['king', 'woman'], negative=['man'], topn=5)

I find the order in which you need to pass the words to the `positive` and `negative` arguments slightly unintuitive, though.
So let's define a wrapper function:

In [ ]:
def analogy(x1='man', y1='king', x2='woman', verbose=True):
    """Computes answer to query 'y1 is to x1 what WORD is to x2?'"""
    result = model.most_similar(positive=[y1, x2], negative=[x1])
    if verbose:
        print(f"'{x1}' : '{y1}' :: '{x2}' : ?? ==> '{result[0][0]}'")
    return result[0][0]

In [ ]:
analogy('man', 'king', 'woman')

### Exercise 3 👩‍💻

Can you come up with analogy problems involving terms from your discipline or research area?
Can the word embedding model solve these specialized problems?

**_Example:_** In politics "Democrat is to progressive what Republican is to ___?"

In [ ]:
v_x1 = model['Democrat']
v_y1 = model['progressive']
v_x2 = model['Republican']

v_q = v_y1 - v_x1 + v_x2

model.similar_by_vector(v_q, topn=5)